In [5]:
df_right['TYPECODE'].value_counts()
df_right['TYPEC']=df_right['TYPECODE'].apply(lambda x:x[0:3])
df_right['TYPEC'].value_counts()
df_right2=df_right[df_right['TYPEC']=='mno']
df_right2.head(10)
df_right2['RIGHTTYPE'].value_counts()

40    386094
Name: RIGHTTYPE, dtype: int64

In [1]:
#coding:utf-8
import pandas as pd
from sklearn.metrics import f1_score,auc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
%matplotlib inline
dir = '../public_fu/'

In [2]:
df_test = pd.read_csv(dir + 'evaluation_public.csv')
df_train = pd.read_csv(dir + 'train.csv')
EID = df_test.copy()
df_entbase = pd.read_csv(dir + '1entbase.csv')
df_entbase = df_entbase.fillna(0)
df_alter = pd.read_csv(dir + '2alter.csv')
df_branch = pd.read_csv(dir + '3branch.csv')
df_invest = pd.read_csv(dir + '4invest.csv')
df_right = pd.read_csv(dir + '5right.csv')
df_project = pd.read_csv(dir + '6project.csv')
df_lawsuit = pd.read_csv(dir + '7lawsuit.csv')
df_breakfaith = pd.read_csv(dir + '8breakfaith.csv')
df_recruit = pd.read_csv(dir + '9recruit.csv')
df_qualification = pd.read_csv(dir + '10qualification.csv')

/home/lixinhui/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/lixinhui/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_branch2 = df_branch.drop_duplicates(['EID','TYPECODE'])
df_invest2 = df_invest.drop_duplicates(['EID','BTEID'])
df_right2 = df_right.drop_duplicates(['EID','TYPECODE'])
df_project2 = df_project.drop_duplicates(['EID','TYPECODE'])
df_lawsuit2 = df_lawsuit.drop_duplicates(['EID','TYPECODE'])
df_breakfaith2 = df_breakfaith.drop_duplicates(['EID','TYPECODE'])
df_branch = df_branch2
df_invest = df_invest2
df_right = df_right2
df_project = df_project2
df_lawsuit = df_lawsuit2
df_breakfaith = df_breakfaith2

In [4]:
def map_alter_num(x):
    if isinstance(x,str):
        if len(x)<4:
            return float(filter(str.isdigit, str(x)))
        else:
            if x[0:2] =='HK':
                return float(x[3:])*0.847
            if x[-2:] == 'HK':
                return float(x[:-3])*0.847
            if x[0:3]=='USA':
                return float(x[4:])*6.2
            if x[-3:] =='USA':
                return float(x[:-4])*6.2
    else:
        return 0
    
def map_alter_type(x): 
    if isinstance(x,str):
        if len(x)<4:
            return 0
        else:
            if x[0:2] =='HK':
                return 1
            if x[-2:] =='HK':
                return 1
            if x[0:3]=='USA':
                return 20
            if x[-3:] =='USA':
                return 20
    else :
        return 0

In [5]:
df_alter['ALTBE_type'] = df_alter['ALTBE'].apply(map_alter_type)
df_alter['ALTBE'] = df_alter['ALTBE'].apply(map_alter_num)
df_alter['ALTAF_type'] = df_alter['ALTAF'].apply(map_alter_type)
df_alter['ALTAF'] = df_alter['ALTAF'].apply(map_alter_num)
df_alter['ALTDATE_YEAR'] = df_alter['ALTDATE'].map(lambda x:x.split('-')[0])
df_alter['ALTDATE_YEAR'] = df_alter['ALTDATE_YEAR'].astype(int)
df_alter['ALTAF_ALTBE'] = df_alter['ALTAF'] - df_alter['ALTBE']
df_alter2014 = df_alter[df_alter['ALTDATE_YEAR']==2014]
df_alter2015 = df_alter[df_alter['ALTDATE_YEAR']==2015]
df_alter_table_type = pd.pivot_table(df_alter,index=['EID'],columns =['ALTERNO'],
                                         values=['ALTBE'],aggfunc={'ALTBE':len})
df_alter_type= pd.DataFrame(index=df_alter_table_type.index,columns=list(df_alter_table_type.columns.levels[1]),data=df_alter_table_type.values)

bad_NO = ['03','10','12','A_015']
df_alter_bad = df_alter[df_alter['ALTERNO'].apply(lambda x: x  in bad_NO)]
df_alter_vector = df_alter_type
df_alter_vector['alter_type_var'] = df_alter_type.var(axis=1)
df_alter_vector['alter_type_skew'] = df_alter_type.skew(axis=1)
df_alter_vector['alter_type_lisan'] = df_alter_type.std(axis=1)*1.0/df_alter_type.mean(axis=1)
df_alter_vector['alter_12_15'] = df_alter_type['12']+df_alter_type['A_015']
df_alter_vector['alter_12_10'] = df_alter_type['12']+df_alter_type['10']
df_alter_vector['alter_ZBchange_sum'] = df_alter.groupby(by=['EID'])['ALTAF_ALTBE'].sum()
df_alter_vector['alter_ZBchange_max'] = df_alter.groupby(by=['EID'])['ALTAF_ALTBE'].max()
df_alter_vector['alter_ZB_mean'] = df_alter.groupby(by=['EID'])['ALTBE'].mean()
df_alter_vector['alter_ZB_max'] = df_alter.groupby(by=['EID'])['ALTBE'].max()
df_alter_vector['alter_bad_count'] = df_alter_bad.groupby(by=['EID'])['ALTDATE'].count()
df_alter_vector['alter_count'] = df_alter.groupby(by=['EID'])['ALTDATE'].count()

df_alter_table_num= pd.pivot_table(df_alter_bad,index=['EID'],columns =['ALTDATE_YEAR'],
                                         values=['ALTERNO'],aggfunc={'ALTERNO':len})
df_alter_num= pd.DataFrame(index=df_alter_table_num.index,columns=list(df_alter_table_num.columns.levels[1]),data=df_alter_table_num.values)

df_alter_table_2014= pd.pivot_table(df_alter2014,index=['EID'],columns =['ALTDATE'],
                                         values=['ALTERNO'],aggfunc={'ALTERNO':len})
df_alter_2014= pd.DataFrame(index=df_alter_table_2014.index,columns=list(df_alter_table_2014.columns.levels[1]),data=df_alter_table_2014.values)

df_alter_table_2015= pd.pivot_table(df_alter2015,index=['EID'],columns =['ALTDATE'],
                                         values=['ALTERNO'],aggfunc={'ALTERNO':len})
df_alter_2015= pd.DataFrame(index=df_alter_table_2015.index,columns=list(df_alter_table_2015.columns.levels[1]),data=df_alter_table_2015.values)

df_alter_vector['alter_2014_max'] = df_alter_2014.max(axis=1)
df_alter_vector['alter_2014_min'] = df_alter_2014.min(axis=1)
df_alter_vector['alter_2014_mean'] = df_alter_2014.mean(axis=1)
df_alter_vector['alter_2014_median'] = df_alter_2014.median(axis=1)
df_alter_vector['alter_2014_max-min'] = df_alter_2014.max(axis=1)-df_alter_2014.min(axis=1)
df_alter_2014 = df_alter_2014.fillna(0)
df_alter_vector['alter_2014_var'] = df_alter_2014.var(axis=1)
df_alter_vector['alter_2014_lisan'] = df_alter_2014.std(axis=1)*1.0/df_alter_2014.mean(axis=1)
df_alter_vector['alter_2014_skewness'] = df_alter_2014.skew(axis=1)
df_alter_vector['alter_2014_kurtosis'] = df_alter_2014.kurtosis(axis=1)

df_alter_vector['alter_2015_max'] = df_alter_2015.max(axis=1)
df_alter_vector['alter_2015_min'] = df_alter_2015.min(axis=1)
df_alter_vector['alter_2015_mean'] = df_alter_2015.mean(axis=1)
df_alter_vector['alter_2015_median'] = df_alter_2015.median(axis=1)
df_alter_vector['alter_2015_max-min'] = df_alter_2015.max(axis=1)-df_alter_2015.min(axis=1)
df_alter_2015 = df_alter_2015.fillna(0)
df_alter_vector['alter_2015_var'] = df_alter_2015.var(axis=1)
df_alter_vector['alter_2015_lisan'] = df_alter_2015.std(axis=1)*1.0/df_alter_2015.mean(axis=1)
df_alter_vector['alter_2015_skewness'] = df_alter_2015.skew(axis=1)
df_alter_vector['alter_2015_kurtosis'] = df_alter_2015.kurtosis(axis=1)

df_alter_vector['alter_bad_mean'] = df_alter_num.mean(axis=1)
df_alter_vector['alter_bad_max'] = df_alter_num.max(axis=1)
df_alter_vector['alter_bad_min'] = df_alter_num.min(axis=1)
df_alter_vector['alter_bad_median'] = df_alter_num.median(axis=1)
df_alter_vector['alter_bad_max-min'] = df_alter_num.max(axis=1)-df_alter_num.min(axis=1)
df_alter_num = df_alter_num.fillna(0)
df_alter_vector['alter_bad_var'] = df_alter_num.var(axis=1)
df_alter_vector['alter_bad_lisan'] = df_alter_num.std(axis=1)*1.0/df_alter_num.mean(axis=1)
df_alter_vector['alter_bad_skewness'] = df_alter_num.skew(axis=1)
#df_alter_vector['alter_bad_kurtosis'] = df_alter_num.kurtosis(axis=1)
df_alter_vector['alter_last_year'] = df_alter.groupby(by=['EID'])['ALTDATE_YEAR'].max()
df_alter_vector['alter_first_year'] = df_alter.groupby(by=['EID'])['ALTDATE_YEAR'].min()
df_alter_vector['alter_year'] =df_alter_vector['alter_last_year'].astype(int) - df_alter.groupby(by=['EID'])['ALTDATE_YEAR'].min().astype(int)
df_alter_vector['alter_bad_rate'] = df_alter_vector['alter_bad_count']*1.0/df_alter_vector['alter_count']
df_alter_vector[['2013_alter','2014_alter','2015_alter']]= df_alter_num
df_alter_vector['alterBE_type'] = df_alter.groupby(by=['EID'])['ALTBE_type'].sum()
df_alter_vector['alterAF_type'] = df_alter.groupby(by=['EID'])['ALTAF_type'].sum()
df_alter_vector.to_csv('../data3/df_alter_vector.csv')

In [6]:
df_alter_vector.head(10)

,01,02,03,04,05,10,12,13,14,27,...,alter_bad_skewness,alter_last_year,alter_first_year,alter_year,alter_bad_rate,2013_alter,2014_alter,2015_alter,alterBE_type,alterAF_type
EID,,,,,,,,,,,,,,,,,,,,,
p10000,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2015,2015,0,NaN,NaN,NaN,NaN,0.0,0.0
p100004,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,2015,2015,0,NaN,NaN,NaN,NaN,0.0,0.0
p100007,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,1.732051,2014,2014,0,1.000000,0.0,1.0,0.0,0.0,0.0
p100010,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,1.732051,2014,2014,0,0.500000,0.0,2.0,0.0,0.0,0.0
p100016,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2015,2015,0,NaN,NaN,NaN,NaN,NaN,NaN
p100018,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1.732051,2015,2015,0,1.000000,0.0,0.0,1.0,0.0,0.0
p100022,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,...,1.732051,2015,2014,1,0.333333,0.0,1.0,0.0,0.0,0.0
p100031,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,...,1.732051,2015,2014,1,0.333333,0.0,1.0,0.0,0.0,0.0
p100034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,2014,2014,0,NaN,NaN,NaN,NaN,0.0,0.0


In [7]:
df_branch['B_ENDYEAR'] = df_branch['B_ENDYEAR'].fillna(3000)
df_branch['B_life'] = df_branch['B_ENDYEAR'] - df_branch['B_REYEAR']
#branch　量化
branch_bin = [0,1945,1979,1992,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,3000]
df_branch['B_ENDYEAR_C'] = pd.cut(df_branch['B_ENDYEAR'],branch_bin,labels=branch_bin[1:])
df_branch['B_REYEAR_C'] = pd.cut(df_branch['B_REYEAR'],branch_bin,labels=branch_bin[1:])

df_table_branch_REYEAR = pd.pivot_table(df_branch,index=['EID'],columns =['B_REYEAR_C'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len},dropna=False)
df_branch_REYEAR= pd.DataFrame(index=df_table_branch_REYEAR.index,
                             columns=list(df_table_branch_REYEAR.columns.levels[1]),data=df_table_branch_REYEAR.values)

df_table_branch_B_ENDYEAR = pd.pivot_table(df_branch,index=['EID'],columns =['B_ENDYEAR_C'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len},dropna=False)
df_branch_B_ENDYEAR= pd.DataFrame(index=df_table_branch_B_ENDYEAR.index,
                             columns=list(df_table_branch_B_ENDYEAR.columns.levels[1]),data=df_table_branch_B_ENDYEAR.values)

df_branch_REYEAR = df_branch_REYEAR.fillna(0)
df_branch_B_ENDYEAR = df_branch_B_ENDYEAR.fillna(0)
df_branch_rate = df_branch_B_ENDYEAR-df_branch_REYEAR
df_branch_rate2 = df_branch_rate.copy()
df_branch_rate['branch_count'] = df_branch.groupby(by=['EID'])['B_life'].count()
df_branch_rate['branch_diff_sum'] = df_branch_rate2.drop([3000],axis=1).sum(axis = 1)
df_branch_rate['branch_diff_max'] = df_branch_rate2.max(axis =1)
df_branch_rate['branch_diff_min'] = df_branch_rate2.min(axis =1)
df_branch_rate['branch_diff_max-min'] = df_branch_rate2.max(axis=1)-df_branch_rate2.min(axis=1)
df_branch_rate['branch_diff_median'] = df_branch_rate2.median(axis =1)
df_branch_rate2 = df_branch_rate2.fillna(0)
df_branch_rate['branch_diff_var'] = df_branch_rate2.var(axis =1)
df_branch_rate['branch_diff_lisan'] = df_branch_rate2.std(axis =1)*1.0/df_branch_rate2.mean(axis =1)
df_branch_rate['branch_diff_skew'] = df_branch_rate2.skew(axis =1)
df_branch_rate['branch_diff_kurtosis'] = df_branch_rate2.kurtosis(axis =1)

df_branch_rate['branch_open_sum'] = df_branch_REYEAR.drop([3000],axis=1).sum(axis = 1)
df_branch_rate['branch_open_max'] = df_branch_REYEAR.max(axis =1)
df_branch_rate['branch_open_min'] = df_branch_REYEAR.min(axis =1)
df_branch_rate['branch_open_median'] = df_branch_REYEAR.median(axis =1)
df_branch_rate['branch_open_max-min'] = df_branch_rate['branch_open_max']-df_branch_rate['branch_open_min']
df_branch_REYEAR = df_branch_REYEAR.fillna(0)
df_branch_rate['branch_open_lisan'] = df_branch_REYEAR.std(axis =1)*1.0/df_branch_REYEAR.mean(axis =1)
df_branch_rate['branch_open_var'] = df_branch_REYEAR.var(axis =1)
df_branch_rate['branch_open_skew'] = df_branch_REYEAR.skew(axis =1)
df_branch_rate['branch_open_kurtosis'] = df_branch_REYEAR.kurtosis(axis =1)

df_branch_rate['b_life_sum'] = df_branch.groupby(by=['EID'])['B_life'].sum()
df_branch_rate['b_life_min'] = df_branch.groupby(by=['EID'])['B_life'].min()
df_branch_rate['b_life_median'] = df_branch.groupby(by=['EID'])['B_life'].median()
df_branch_rate['b_life_max'] = df_branch.groupby(by=['EID'])['B_life'].max()
#branch　量化
branch_bin = [0,1979,1992,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
df_branch['B_ENDYEAR_C'] = pd.cut(df_branch['B_ENDYEAR'],branch_bin,labels=branch_bin[1:])
df_table_branch_B_ENDYEAR = pd.pivot_table(df_branch,index=['EID'],columns =['B_ENDYEAR_C'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len},dropna=False)
df_branch_B_ENDYEAR2= pd.DataFrame(index=df_table_branch_B_ENDYEAR.index,
                             columns=list(df_table_branch_B_ENDYEAR.columns.levels[1]),data=df_table_branch_B_ENDYEAR.values)
df_branch_rate['branch_END_sum'] = df_branch_B_ENDYEAR2.sum(axis=1)
df_branch_rate['branch_END_max'] = df_branch_B_ENDYEAR2.max(axis=1)
df_branch_rate['branch_END_min'] = df_branch_B_ENDYEAR2.min(axis=1)
df_branch_rate['branch_END_median'] = df_branch_B_ENDYEAR2.median(axis =1)
df_branch_rate['branch_END_max-min'] = df_branch_rate['branch_END_max']-df_branch_rate['branch_END_min']
df_branch_rate['branch_END_lisan'] = df_branch_B_ENDYEAR2.std(axis =1)*1.0/df_branch_B_ENDYEAR2.mean(axis =1)
df_branch_B_ENDYEAR2 = df_branch_B_ENDYEAR2.fillna(0)
df_branch_rate['branch_END_var'] = df_branch_B_ENDYEAR2.var(axis=1)
df_branch_rate['branch_END_skew'] = df_branch_B_ENDYEAR2.skew(axis=1)
df_branch_rate['branch_END_kurtosis'] = df_branch_B_ENDYEAR2.kurtosis(axis=1)
df_branch_rate['branch_END_year'] = df_branch.groupby(by=['EID'])['B_ENDYEAR_C'].max()-df_branch.groupby(by=['EID'])['B_REYEAR_C'].max()  
df_branch_rate['branch_last_year'] = df_branch.groupby(by=['EID'])['B_REYEAR_C'].max()
df_branch_rate['branch_first_year'] = df_branch.groupby(by=['EID'])['B_REYEAR_C'].min()
df_branch_rate['branch_year']=df_branch_rate['branch_last_year'].astype(int)- df_branch.groupby(by=['EID'])['B_REYEAR_C'].min().astype(int)
df_branch_rate['b_no_home'] = df_branch.groupby(by=['EID'])['IFHOME'].count()-df_branch.groupby(by=['EID'])['IFHOME'].sum()
df_branch_rate['b_home_rate'] = df_branch.groupby(by=['EID'])['IFHOME'].sum()*1.0/df_branch.groupby(by=['EID'])['IFHOME'].count()
df_branch_rate['branch_end_rate'] = df_branch_rate['branch_END_sum']*1.0/df_branch_rate['branch_count']
df_branch_rate.to_csv('../data3/df_branch_vector.csv')

In [8]:
df_branch_rate.head(10)

,1945,1979,1992,2004,2005,2006,2007,2008,2009,2010,...,branch_END_var,branch_END_skew,branch_END_kurtosis,branch_END_year,branch_last_year,branch_first_year,branch_year,b_no_home,b_home_rate,branch_end_rate
EID,,,,,,,,,,,,,,,,,,,,,
p100035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2014,2014,0,0,1.0,NaN
p100037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2013,2013,0,0,1.0,NaN
p100118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,...,0.071429,3.741657,14.0,3.0,2008,2008,0,0,1.0,0.500000
p100164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.071429,3.741657,14.0,0.0,2015,2010,5,0,1.0,0.052632
p100187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2014,2013,1,0,1.0,NaN
p100205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2008,2008,0,0,1.0,NaN
p100206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2015,2012,3,0,1.0,NaN
p100230,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.071429,3.741657,14.0,6.0,2006,2006,0,0,1.0,1.000000
p100266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.071429,3.741657,14.0,3.0,2008,2008,0,0,1.0,1.000000


In [9]:
df_invest['BTENDYEAR'] = df_invest['BTENDYEAR'].fillna(3000)
df_invest['I_life'] = df_invest['BTENDYEAR'] - df_invest['BTYEAR']

#invest　量化
invest_bin = [0,1945,1979,1992,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,3000]
df_invest['BTENDYEAR_C'] = pd.cut(df_invest['BTENDYEAR'],invest_bin,labels=invest_bin[1:])
df_invest['BTYEAR_C'] = pd.cut(df_invest['BTYEAR'],invest_bin,labels=invest_bin[1:])

df_table_invest_REYEAR = pd.pivot_table(df_invest,index=['EID'],columns =['BTYEAR_C'],
                                         values=['BTEID'],aggfunc={'BTEID':len},dropna=False)
df_invest_REYEAR= pd.DataFrame(index=df_table_invest_REYEAR.index,
                             columns=list(df_table_invest_REYEAR.columns.levels[1]),data=df_table_invest_REYEAR.values)

df_table_invest_B_ENDYEAR = pd.pivot_table(df_invest,index=['EID'],columns =['BTENDYEAR_C'],
                                         values=['BTEID'],aggfunc={'BTEID':len},dropna=False)
df_invest_B_ENDYEAR= pd.DataFrame(index=df_table_invest_B_ENDYEAR.index,
                             columns=list(df_table_invest_B_ENDYEAR.columns.levels[1]),data=df_table_invest_B_ENDYEAR.values)
df_invest_REYEAR = df_invest_REYEAR.fillna(0)
df_invest_B_ENDYEAR = df_invest_B_ENDYEAR.fillna(0)
df_invest_rate = df_invest_B_ENDYEAR-df_invest_REYEAR
df_invest_rate2 = df_invest_rate.copy()
df_invest_rate['invest_count'] = df_invest.groupby(by=['EID'])['BTBL'].count()
df_invest_rate['invest_diff_sum'] = df_invest_rate2.drop([3000],axis=1).sum(axis = 1)
df_invest_rate['invest_diff_var'] = df_invest_rate2.var(axis =1)
df_invest_rate['invest_diff_max'] = df_invest_rate2.max(axis =1)
df_invest_rate['invest_diff_min'] = df_invest_rate2.min(axis =1)
df_invest_rate['invest_diff_max-min'] = df_invest_rate2.max(axis=1)-df_invest_rate2.min(axis=1)
df_invest_rate['invest_diff_median'] = df_invest_rate2.median(axis =1)
df_invest_rate2 = df_invest_rate2.fillna(0)
df_invest_rate['invest_diff_var'] = df_invest_rate2.var(axis =1)
df_invest_rate['invest_diff_lisan'] = df_invest_rate2.std(axis =1)*1.0/df_invest_rate2.mean(axis =1)
df_invest_rate['invest_diff_skew'] = df_invest_rate2.skew(axis =1)
df_invest_rate['invest_diff_kurtosis'] = df_invest_rate2.kurtosis(axis =1)

df_invest_rate['invest_open_sum'] = df_invest_REYEAR.drop([3000],axis=1).sum(axis = 1)
df_invest_rate['invest_open_max'] = df_invest_REYEAR.max(axis =1)
df_invest_rate['invest_open_min'] = df_invest_REYEAR.min(axis =1)
df_invest_rate['invest_open_median'] = df_invest_REYEAR.median(axis =1)
df_invest_rate['invest_open_max-min'] = df_invest_REYEAR.max(axis =1)-df_invest_REYEAR.min(axis =1)   
df_invest_REYEAR = df_invest_REYEAR.fillna(0)                                                                  
df_invest_rate['invest_open_var'] = df_invest_REYEAR.var(axis =1)
df_invest_rate['invest_open_lisan'] = df_invest_REYEAR.std(axis =1)*1.0/df_invest_REYEAR.mean(axis =1)
df_invest_rate['invest_open_skew'] = df_invest_REYEAR.skew(axis =1)                                                                  
df_invest_rate['invest_open_kurtosis'] = df_invest_REYEAR.kurtosis(axis =1)
                                                                  
df_invest_rate['I_life_sum'] = df_invest.groupby(by=['EID'])['I_life'].sum()
df_invest_rate['I_life_min'] = df_invest.groupby(by=['EID'])['I_life'].min()
df_invest_rate['I_life_max'] = df_invest.groupby(by=['EID'])['I_life'].max()
df_invest_rate['I_life_median'] = df_invest.groupby(by=['EID'])['I_life'].median()
                                                                  
df_invest_rate['invest_BTBL_sum'] = df_invest.groupby(by=['EID'])['BTBL'].sum()
df_invest_rate['invest_BTBL_rate'] = df_invest.groupby(by=['EID'])['BTBL'].sum()*1.0/df_invest.groupby(by=['EID'])['BTBL'].count()
df_invest_rate['invest_BTBL_max'] = df_invest.groupby(by=['EID'])['BTBL'].max()
df_invest_rate['invest_BTBL_min'] = df_invest.groupby(by=['EID'])['BTBL'].min()
#invest　量化
invest_bin = [0,1979,1992,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
df_invest['BTENDYEAR_C'] = pd.cut(df_invest['BTENDYEAR'],invest_bin,labels=invest_bin[1:])
df_table_invest_B_ENDYEAR = pd.pivot_table(df_invest,index=['EID'],columns =['BTENDYEAR_C'],
                                         values=['BTEID'],aggfunc={'BTEID':len},dropna=False)
df_invest_B_ENDYEAR2= pd.DataFrame(index=df_table_invest_B_ENDYEAR.index,
                             columns=list(df_table_invest_B_ENDYEAR.columns.levels[1]),data=df_table_invest_B_ENDYEAR.values)

df_invest_rate['invest_END_sum'] = df_invest_B_ENDYEAR2.sum(axis=1)
df_invest_rate['invest_END_max'] = df_invest_B_ENDYEAR2.max(axis=1)
df_invest_rate['invest_END_min'] = df_invest_B_ENDYEAR2.min(axis=1)
df_invest_rate['invest_END_median'] = df_invest_B_ENDYEAR2.median(axis=1)
df_invest_rate['invest_END_max-min'] = df_invest_B_ENDYEAR2.max(axis=1) - df_invest_B_ENDYEAR2.min(axis=1)                                                               
df_invest_B_ENDYEAR2=df_invest_B_ENDYEAR2.fillna(0)                                                                                                                                  
df_invest_rate['invest_END_var'] = df_invest_B_ENDYEAR2.var(axis=1)
df_invest_rate['invest_END_lisan'] = df_invest_B_ENDYEAR2.std(axis=1)*1.0/df_invest_B_ENDYEAR2.mean(axis=1)                                                         
df_invest_rate['invest_END_skew'] = df_invest_B_ENDYEAR2.skew(axis=1) 
df_invest_rate['invest_END_kurtosis'] = df_invest_B_ENDYEAR2.kurtosis(axis=1)                                                                  
                                                                  
df_invest_rate['invest_END_year'] = df_invest.groupby(by=['EID'])['BTENDYEAR_C'].max()-df_invest.groupby(by=['EID'])['BTYEAR_C'].max()  
df_invest_rate['invest_last_year'] = df_invest.groupby(by=['EID'])['BTYEAR_C'].max()
df_invest_rate['invest_first_year'] = df_invest.groupby(by=['EID'])['BTYEAR_C'].min()
df_invest_rate['invest_year']= df_invest_rate['invest_last_year'].astype(int)- df_invest.groupby(by=['EID'])['BTYEAR_C'].min().astype(int)
df_invest_rate['I_no_home'] = df_invest.groupby(by=['EID'])['IFHOME'].count()-df_invest.groupby(by=['EID'])['IFHOME'].sum()
df_invest_rate['I_home_rate'] = df_invest.groupby(by=['EID'])['IFHOME'].sum()*1.0/df_invest.groupby(by=['EID'])['IFHOME'].count()
df_invest_rate['invest_end_rate'] = df_invest_rate['invest_END_sum']*1.0/df_invest_rate['invest_count']

df_invest_rate.to_csv('../data3/df_invest_vector.csv')

In [10]:
def map_DATE_NULL(x):
    if isinstance(x,str):
        if x == 'NAN':
            return -1
        else:
            return x.split('-')[0]
    else:
        return -1

In [11]:
df_table_right_type = pd.pivot_table(df_right,index=['EID'],columns =['RIGHTTYPE'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len})
df_right_type = pd.DataFrame(index=df_table_right_type.index,columns=list(df_table_right_type.columns.levels[1]),data=df_table_right_type.values)

df_right['ASKDATE'] = df_right['ASKDATE'].apply(lambda x:x.split('-')[0])   
df_right['FBDATE'] = df_right['FBDATE'].apply(map_DATE_NULL) 

right_bin = [0,1945,1979,1992,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
df_right['ASKDATE_C'] = pd.cut(df_right['ASKDATE'].astype(int),right_bin,labels=right_bin[1:])
df_right['FBDATE_C'] = pd.cut(df_right['FBDATE'].astype(int),right_bin,labels=right_bin[1:])

df_right_ASK = pd.pivot_table(df_right,index=['EID'],columns =['ASKDATE_C'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len},dropna=False)
df_right_ASK_date = pd.DataFrame(index=df_right_ASK.index,columns=list(df_right_ASK.columns.levels[1]),data=df_right_ASK.values)

df_right_FBDATE = pd.pivot_table(df_right,index=['EID'],columns =['FBDATE_C'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len},dropna=False)
df_right_FBDATE_date = pd.DataFrame(index=df_right_FBDATE.index,columns=list(df_right_FBDATE.columns.levels[1]),data=df_right_FBDATE.values)

df_right_type = df_right_type.fillna(0)
df_right_vector = df_right_type.copy()
df_right_vector['right_type_var'] = df_right_type.var(axis=1)
df_right_vector['right_type_skew'] = df_right_type.skew(axis=1)
df_right_vector['right_type_kurtosis'] = df_right_type.kurtosis(axis=1)

df_right_vector['right_count'] = df_right.groupby(by=['EID'])['TYPECODE'].count()
df_right_vector['right_FBDATE_count'] = df_right.groupby(by=['EID'])['FBDATE'].count()

df_right_ASK_date = df_right_ASK_date.fillna(0)
df_right_ASK_date_diff = df_right_ASK_date.diff(axis=1)
df_right_FBDATE_date = df_right_FBDATE_date.fillna(0)
df_right_FBDATE_date_diff = df_right_FBDATE_date.diff(axis=1)


df_right_vector['right_ASKDATE_max'] = df_right_ASK_date.max(axis=1)
df_right_vector['right_ASKDATE_min'] = df_right_ASK_date.min(axis=1)
df_right_vector['right_ASKDATE_median'] = df_right_ASK_date.median(axis=1)
df_right_vector['right_ASKDATE_max-min'] = df_right_ASK_date.max(axis=1)-df_right_ASK_date.min(axis=1)

df_right_vector['right_ASKDATE_var'] = df_right_ASK_date.var(axis=1)
df_right_vector['right_ASKDATE_lisan'] = df_right_ASK_date.std(axis=1)*1.0/df_right_ASK_date.mean(axis=1)
df_right_vector['right_ASKDATE_skew'] = df_right_ASK_date.skew(axis=1)
df_right_vector['right_ASKDATE_kurtosis'] = df_right_ASK_date.kurtosis(axis=1)

df_right_vector['right_FBDATE_max'] = df_right_FBDATE_date.max(axis=1)
df_right_vector['right_FBDATE_min'] = df_right_FBDATE_date.min(axis=1)
df_right_vector['right_FBDATE_median'] = df_right_FBDATE_date.median(axis=1)
df_right_vector['right_FBDATE_max-min'] = df_right_FBDATE_date.max(axis=1)-df_right_FBDATE_date.min(axis=1)

df_right_vector['right_FBDATE_var'] = df_right_FBDATE_date.var(axis=1)
df_right_vector['right_FBDATE_lisan'] = df_right_FBDATE_date.std(axis=1)*1.0/df_right_FBDATE_date.mean(axis=1)
df_right_vector['right_FBDATE_skew'] = df_right_FBDATE_date.skew(axis=1)
df_right_vector['right_FBDATE_kurtosis'] = df_right_FBDATE_date.kurtosis(axis=1)

df_right_vector['right_ASKDATE_diff_sum'] = df_right_ASK_date_diff.sum(axis=1)
df_right_vector['right_FBDATE_diff_sum'] = df_right_FBDATE_date_diff.sum(axis=1)
df_right_vector['right_ASKDATE_diff_max'] = df_right_ASK_date_diff.max(axis=1)

df_right_vector['right_2007'] = df_right_ASK_date_diff[2007]
df_right_vector['right_2008'] = df_right_ASK_date_diff[2008]
df_right_vector['right_2009'] = df_right_ASK_date_diff[2009]
df_right_vector['right_2010'] = df_right_ASK_date_diff[2010]
df_right_vector['right_2011'] = df_right_ASK_date_diff[2011]
df_right_vector['right_2012'] = df_right_ASK_date_diff[2012]
df_right_vector['right_2013'] = df_right_ASK_date_diff[2013]
df_right_vector['right_2014'] = df_right_ASK_date_diff[2014]
df_right_vector['right_2015'] = df_right_ASK_date_diff[2015]
#最近5年的　方差　总数　差分和
right_bin3 = [2010,2011,2012,2013,2014,2015]
import_right = [11, 12, 20, 30, 50]
df_right2 = df_right[df_right['RIGHTTYPE'].apply(lambda x : x in import_right)]
df_right2['ASKDATE_D'] = pd.cut(df_right2['ASKDATE'].astype(int),right_bin3,labels=right_bin3[1:])
df_right2['FBDATE_D'] = pd.cut(df_right2['FBDATE'].astype(int),right_bin3,labels=right_bin3[1:])

df_right_ASK = pd.pivot_table(df_right2,index=['EID'],columns =['ASKDATE_D'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len},dropna=False)
df_right_ASK_date3 = pd.DataFrame(index=df_right_ASK.index,columns=list(df_right_ASK.columns.levels[1]),data=df_right_ASK.values)

df_right_FBDATE = pd.pivot_table(df_right2,index=['EID'],columns =['FBDATE_D'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len},dropna=False)
df_right_FBDATE_date3 = pd.DataFrame(index=df_right_FBDATE.index,columns=list(df_right_FBDATE.columns.levels[1]),data=df_right_FBDATE.values)

df_right_ASK_date3 = df_right_ASK_date3.fillna(0)
df_right_ASK_date_diff = df_right_ASK_date3.diff(axis=1)
df_right_FBDATE_date3 = df_right_FBDATE_date3.fillna(0)
df_right_FBDATE_date_diff = df_right_FBDATE_date3.diff(axis=1)

df_right_vector['right_ASKDATE_var5'] = df_right_ASK_date3.var(axis=1)
df_right_vector['right_FBDATE_var5'] = df_right_FBDATE_date3.var(axis=1)

df_right_vector['right_ASKDATE_skew5'] = df_right_ASK_date3.skew(axis=1)
df_right_vector['right_FBDATE_skew5'] = df_right_FBDATE_date3.skew(axis=1)

df_right_vector['right_ASKDATE_lisan5'] = df_right_ASK_date3.std(axis=1)*1.0/df_right_ASK_date3.mean(axis=1)
df_right_vector['right_FBDATE_lisan5'] = df_right_FBDATE_date3.std(axis=1)*1.0/df_right_FBDATE_date3.mean(axis=1)

df_right_vector['right_ASKDATE_max5'] = df_right_ASK_date3.max(axis=1)
df_right_vector['right_FBDATE_max5'] = df_right_FBDATE_date3.max(axis=1)
df_right_vector['right_ASKDATE_sum5'] = df_right_ASK_date3.sum(axis=1)
df_right_vector['right_FBDATE_sum5'] = df_right_FBDATE_date3.sum(axis=1)

df_right_vector['right_ASKDATE_diff3_sum'] = df_right_ASK_date_diff.sum(axis=1)
df_right_vector['right_FBDATE_diff3_sum'] = df_right_FBDATE_date_diff.sum(axis=1)


df_right_vector['right_last_year'] = df_right.groupby(by=['EID'])['ASKDATE_C'].max()
df_right_vector['right_first_year'] = df_right.groupby(by=['EID'])['ASKDATE_C'].min()
df_right_vector['right_year']= df_right_vector['right_last_year'].astype(int)- df_right.groupby(by=['EID'])['ASKDATE_C'].min().astype(int)

df_right_vector.to_csv('../data3/df_right_vector.csv')

/home/lixinhui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/lixinhui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
df_invest_rate.head(10)

,1945,1979,1992,2004,2005,2006,2007,2008,2009,2010,...,invest_END_lisan,invest_END_skew,invest_END_kurtosis,invest_END_year,invest_last_year,invest_first_year,invest_year,I_no_home,I_home_rate,invest_end_rate
EID,,,,,,,,,,,,,,,,,,,,,
p100164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2015,2015,0,0,1.0,NaN
p100184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2013,2013,0,0,1.0,NaN
p10019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2015,2014,1,2,0.6,NaN
p100206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.741657,3.741657,14.0,0.0,2014,2012,2,0,1.0,0.500000
p100217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2015,2015,0,0,1.0,NaN
p100220,0.0,0.0,0.0,-2.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.741657,3.741657,14.0,-4.0,2011,2004,7,0,1.0,0.333333
p100234,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2004,1992,12,0,1.0,NaN
p100271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2011,2011,0,0,1.0,NaN
p100283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,2014,2011,3,0,1.0,NaN


In [13]:
df_right_vector.head(10)

,11,12,20,30,40,50,60,right_type_var,right_type_skew,right_type_kurtosis,...,right_FBDATE_lisan5,right_ASKDATE_max5,right_FBDATE_max5,right_ASKDATE_sum5,right_FBDATE_sum5,right_ASKDATE_diff3_sum,right_FBDATE_diff3_sum,right_last_year,right_first_year,right_year
EID,,,,,,,,,,,,,,,,,,,,,
p100000,0.0,0.0,0.0,0.0,7.0,0.0,2.0,6.904762,2.267626,5.172842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2012,3
p100001,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.142857,2.645751,7.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,2013,0
p10001,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.285714,2.645751,7.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015,0
p100014,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.142857,2.645751,7.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,2012,0
p100022,0.0,0.0,0.0,0.0,4.0,0.0,5.0,4.904762,1.301998,-0.389952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,2014,0
p100028,0.0,0.0,0.0,0.0,3.0,0.0,1.0,1.285714,2.155797,4.580247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,2014,0
p10003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.142857,2.645751,7.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,2010,0
p100031,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.142857,2.645751,7.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,2012,0
p100036,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.142857,2.645751,7.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015,0


In [14]:
df_project['DJDATE_Y'] = df_project['DJDATE'].apply(lambda x:x.split('-')[0]) 
df_project_table_DATE= pd.pivot_table(df_project,index=['EID'],columns =['DJDATE_Y'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len})
df_project_DATE = pd.DataFrame(index=df_project_table_DATE.index,columns=list(df_project_table_DATE.columns.levels[1]),data=df_project_table_DATE.values)
df_project_DATE = df_project_DATE.fillna(0)
df_project_DATE_diff = df_project_DATE.diff(axis=1)
df_project_vector = df_project_DATE_diff.dropna(how='all',axis=1)
df_project_vector['project_first_year'] = df_project.groupby(by=['EID'])['DJDATE_Y'].min()
df_project_vector['project_last_year'] = df_project.groupby(by=['EID'])['DJDATE_Y'].max()
df_project_vector['project_year'] = df_project_vector['project_last_year'].astype(int) - df_project.groupby(by=['EID'])['DJDATE_Y'].min().astype(int)
df_project_vector['project_max'] = df_project_DATE.max(axis=1)
df_project_vector['project_min'] = df_project_DATE.min(axis=1)
df_project_vector['project_median'] = df_project_DATE.median(axis=1)
df_project_vector['project_max-min'] = df_project_DATE.max(axis=1)- df_project_DATE.min(axis=1)
df_project_vector['project_lisan'] = df_project_DATE.std(axis=1)*1.0/df_project_DATE.mean(axis=1)
df_project_vector['project_sum'] = df_project.groupby(by=['EID'])['DJDATE_Y'].count()
df_project_vector['project_no_home'] = df_project.groupby(by=['EID'])['IFHOME'].count()-df_project.groupby(by=['EID'])['IFHOME'].sum()
df_project_vector['project_home_rate'] = -df_project.groupby(by=['EID'])['IFHOME'].sum()*1.0/df_project.groupby(by=['EID'])['IFHOME'].count()
df_project_vector['project_last_and_no_home'] = df_project_vector['project_last_year'].astype(int)+df_project_vector['project_no_home']
df_project_vector.to_csv('../data3/df_project_vector.csv')

In [15]:
df_project_vector.head(10)

,2014,2015,project_first_year,project_last_year,project_year,project_max,project_min,project_median,project_max-min,project_lisan,project_sum,project_no_home,project_home_rate,project_last_and_no_home
EID,,,,,,,,,,,,,,
p100098,0.0,1.0,2015,2015,0,1.0,0.0,0.0,1.0,1.732051,1,0,-1.0,2015
p100187,0.0,1.0,2015,2015,0,1.0,0.0,0.0,1.0,1.732051,1,0,-1.0,2015
p100314,0.0,2.0,2015,2015,0,2.0,0.0,0.0,2.0,1.732051,2,0,-1.0,2015
p100435,1.0,-1.0,2014,2014,0,1.0,0.0,0.0,1.0,1.732051,1,0,-1.0,2014
p100450,3.0,0.0,2014,2015,1,3.0,0.0,3.0,3.0,0.866025,6,0,-1.0,2015
p100538,-4.0,-2.0,2013,2015,2,7.0,1.0,3.0,6.0,0.833196,11,0,-1.0,2015
p100548,0.0,1.0,2015,2015,0,1.0,0.0,0.0,1.0,1.732051,1,0,-1.0,2015
p100588,1.0,-1.0,2014,2014,0,1.0,0.0,0.0,1.0,1.732051,1,0,-1.0,2014
p100847,0.0,1.0,2015,2015,0,1.0,0.0,0.0,1.0,1.732051,1,0,-1.0,2015


In [16]:
df_lawsuit['LAWAMOUNT'] = df_lawsuit['LAWAMOUNT'].fillna(0.1)
df_lawsuit['LAWDATE_Y'] = df_lawsuit['LAWDATE'].apply(lambda x:x.split('-')[0])  

df_lawsuit_table = pd.pivot_table(df_lawsuit,index=['EID'],columns =['LAWDATE_Y'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len})
df_lawsuit_DATE = pd.DataFrame(index=df_lawsuit_table.index,columns=list(df_lawsuit_table.columns.levels[1]),data=df_lawsuit_table.values)
df_lawsuit_DATE =df_lawsuit_DATE.fillna(0)
df_lawsuit_DATE2 =df_lawsuit_DATE.copy()
df_lawsuit_DATE['lawsuit_var'] = df_lawsuit_DATE2.var(axis=1)
df_lawsuit_DATE['lawsuit_median'] = df_lawsuit_DATE2.median(axis=1)
df_lawsuit_DATE['lawsuit_lisan'] = df_lawsuit_DATE2.std(axis=1)*1.0/df_lawsuit_DATE2.mean(axis=1)
df_lawsuit_DATE['lawsuit_count'] = df_lawsuit.groupby(by=['EID'])['LAWAMOUNT'].count()
df_lawsuit_DATE['lawsuit_last_year'] = df_lawsuit.groupby(by=['EID'])['LAWDATE_Y'].max()
df_lawsuit_DATE['lawsuit_first_year'] = df_lawsuit.groupby(by=['EID'])['LAWDATE_Y'].min()
df_lawsuit_DATE['lawsuit_year'] = df_lawsuit_DATE['lawsuit_last_year'].astype(int)-df_lawsuit.groupby(
                                   by=['EID'])['LAWDATE_Y'].min().astype(int)
df_lawsuit_DATE['lawsuit_LAWA_sum']=df_lawsuit.groupby(by=['EID'])['LAWAMOUNT'].sum()
df_lawsuit_DATE['lawsuit_LAWA_max']=df_lawsuit.groupby(by=['EID'])['LAWAMOUNT'].max()
df_lawsuit_DATE['lawsuit_LAWA_mean']=df_lawsuit.groupby(by=['EID'])['LAWAMOUNT'].mean()
df_lawsuit_DATE.to_csv('../data3/df_lawsuit_vector.csv')

In [17]:
df_lawsuit_DATE.head(10)

,2012,2013,2014,2015,lawsuit_var,lawsuit_median,lawsuit_lisan,lawsuit_count,lawsuit_last_year,lawsuit_first_year,lawsuit_year,lawsuit_LAWA_sum,lawsuit_LAWA_max,lawsuit_LAWA_mean
EID,,,,,,,,,,,,,,
p100022,0.0,0.0,2.0,0.0,1.000000,0.0,2.000000,2,2014,2014,0,122000,61000,6.100000e+04
p100048,0.0,0.0,2.0,0.0,1.000000,0.0,2.000000,2,2014,2014,0,22000,11000,1.100000e+04
p100074,0.0,0.0,0.0,3.0,2.250000,0.0,2.000000,3,2015,2015,0,8646200,5509600,2.882067e+06
p100084,0.0,0.0,2.0,0.0,1.000000,0.0,2.000000,2,2014,2014,0,1488400,744200,7.442000e+05
p100107,0.0,2.0,0.0,0.0,1.000000,0.0,2.000000,2,2013,2013,0,2,1,1.000000e+00
p100129,1.0,0.0,0.0,0.0,0.250000,0.0,2.000000,1,2012,2012,0,118900,118900,1.189000e+05
p100165,0.0,0.0,0.0,1.0,0.250000,0.0,2.000000,1,2015,2015,0,51100,51100,5.110000e+04
p100186,0.0,0.0,0.0,1.0,0.250000,0.0,2.000000,1,2015,2015,0,26700,26700,2.670000e+04
p100253,1.0,0.0,0.0,10.0,23.583333,0.5,1.765915,11,2015,2012,3,3002800,3000000,2.729818e+05


In [18]:
df_breakfaith['SXENDDATE'] = df_breakfaith['SXENDDATE'].fillna(0)
df_breakfaith['is_breakfaith'] = df_breakfaith['SXENDDATE']!=0
df_breakfaith['is_breakfaith'] = df_breakfaith['is_breakfaith'].astype(int)

df_breakfaith['FBDATE_Y'] = df_breakfaith['FBDATE'].apply(lambda x:x.split('-')[0])  


df_breakfaith_table = pd.pivot_table(df_breakfaith,index=['EID'],columns =['FBDATE_Y'],
                                         values=['TYPECODE'],aggfunc={'TYPECODE':len})
df_breakfaith_DATE = pd.DataFrame(index=df_breakfaith_table.index,columns=list(df_breakfaith_table.columns.levels[1]),data=df_breakfaith_table.values)

df_breakfaith_DATE2 = df_breakfaith_DATE.fillna(0)
df_breakfaith_DATE['breakfaith_median'] = df_breakfaith_DATE.median(axis=1)
df_breakfaith_DATE['breakfaith_max'] = df_breakfaith_DATE.max(axis=1)
df_breakfaith_DATE['breakfaith_lisan'] = df_breakfaith_DATE.std(axis=1)*1.0/df_breakfaith_DATE.mean(axis=1)
df_breakfaith_DATE['var'] = df_breakfaith_DATE.var(axis=1)
df_breakfaith_DATE['breakfaith_count'] = df_breakfaith.groupby(by=['EID'])['FBDATE'].count()
df_breakfaith_DATE['breakfaith_last_year'] = df_breakfaith.groupby(by=['EID'])['FBDATE_Y'].max()
df_breakfaith_DATE['breakfaith_first_year'] = df_breakfaith.groupby(by=['EID'])['FBDATE_Y'].min()
df_breakfaith_DATE['is_breakfaith_sum'] = df_breakfaith.groupby(by=['EID'])['is_breakfaith'].sum()
df_breakfaith_DATE['is_breakfaith_count'] = df_breakfaith.groupby(by=['EID'])['is_breakfaith'].count()
df_breakfaith_DATE['is_breakfaith_rate'] = df_breakfaith_DATE['is_breakfaith_sum']*1.0/df_breakfaith_DATE['is_breakfaith_count']
df_breakfaith_DATE['breakfaith_year'] = df_breakfaith_DATE['breakfaith_last_year'].astype(int)-df_breakfaith.groupby(
                                   by=['EID'])['FBDATE_Y'].min().astype(int)
#df_breakfaith_DATE['breakfaith_year_add'] = df_breakfaith_DATE['breakfaith_year'].astype(int)+df_breakfaith_DATE['breakfaith_count']
df_breakfaith_DATE.to_csv('../data3/df_breakfaith_vector.csv')

In [19]:
df_breakfaith_DATE.head()

,2013,2014,2015,breakfaith_median,breakfaith_max,breakfaith_lisan,var,breakfaith_count,breakfaith_last_year,breakfaith_first_year,is_breakfaith_sum,is_breakfaith_count,is_breakfaith_rate,breakfaith_year
EID,,,,,,,,,,,,,,
p100048,NaN,NaN,1.0,1.0,1.0,0.0,0.25,1,2015,2015,1,1,1.0,0
p100344,NaN,NaN,8.0,8.0,8.0,0.0,16.00,8,2015,2015,0,8,0.0,0
p100347,NaN,NaN,10.0,10.0,10.0,0.0,25.00,10,2015,2015,0,10,0.0,0
p100667,NaN,NaN,2.0,2.0,2.0,0.0,1.00,2,2015,2015,0,2,0.0,0
p101171,NaN,NaN,2.0,2.0,2.0,0.0,1.00,2,2015,2015,0,2,0.0,0


In [20]:
def map_recruit_num(x):
    if isinstance(x,str):
        return int(filter(str.isdigit,x))
    else:
        return 0

In [21]:
df_recruit['RECDATE_Y'] = df_recruit['RECDATE'].apply(lambda x:x.split('-')[0])
df_recruit['PNUM'] = df_recruit['PNUM'].apply(map_recruit_num)
df_recruit_table_DATE= pd.pivot_table(df_recruit,index=['EID'],columns =['RECDATE_Y'],
                                         values=['RECDATE'],aggfunc={'RECDATE':len})
df_recruit_DATE = pd.DataFrame(index=df_recruit_table_DATE.index,columns=list(df_recruit_table_DATE.columns.levels[1]),data=df_recruit_table_DATE.values)

df_recruit_table= pd.pivot_table(df_recruit,index=['EID'],columns =['WZCODE'],
                                         values=['RECDATE'],aggfunc={'RECDATE':len})
df_recruit_web = pd.DataFrame(index=df_recruit_table.index,columns=list(df_recruit_table.columns.levels[1]),data=df_recruit_table.values)
df_recruit_DATE = df_recruit_DATE.fillna(0)
df_recruit_DATE2 = df_recruit_DATE.copy()
df_recruit_DATE['recruit_mean'] = df_recruit_DATE2.mean(axis=1)
df_recruit_DATE['recruit_lisan'] = df_recruit_DATE2.std(axis=1)*1.0/df_recruit_DATE2.mean(axis=1)
df_recruit_vector = df_recruit_web.copy() 
df_recruit_DATE['recruit_web'] = df_recruit_web.var(axis=1)
df_recruit_vector = df_recruit_vector.join(df_recruit_DATE,how = 'left')
df_recruit_vector['recruit_last_year'] =df_recruit.groupby(by=['EID'])['RECDATE_Y'].max()
df_recruit_vector['recruit_first_year'] =df_recruit.groupby(by=['EID'])['RECDATE_Y'].min()
df_recruit_vector['recruit_year'] = df_recruit_vector['recruit_last_year'].astype(int)- df_recruit.groupby(by=['EID'])['RECDATE_Y'].min().astype(int)
df_recruit_vector['recruit_diff'] = df_recruit_DATE.var(axis=1)
df_recruit_vector['recruit_count'] = df_recruit.groupby(by=['EID'])['RECDATE_Y'].count()
df_recruit_vector['recruit_num'] = df_recruit.groupby(by=['EID'])['PNUM'].sum(axis=1)
df_recruit_vector.to_csv('../data3/df_recruit_vector.csv')

In [22]:
df_recruit_vector.head()

,zp01,zp02,zp03,2008,2012,2013,2014,2015,recruit_mean,recruit_lisan,recruit_web,recruit_last_year,recruit_first_year,recruit_year,recruit_diff,recruit_count,recruit_num
EID,,,,,,,,,,,,,,,,,
p100004,NaN,3.0,6.0,0.0,0.0,0.0,5.0,4.0,1.8,1.383322,4.500000,2015,2014,1,4.515989,9,0
p100008,4.0,NaN,5.0,0.0,0.0,0.0,4.0,5.0,1.8,1.383322,0.500000,2015,2014,1,3.756464,9,0
p100032,NaN,NaN,4.0,0.0,0.0,0.0,4.0,0.0,0.8,2.236068,NaN,2014,2014,0,2.427946,4,0
p100037,1.0,3.0,7.0,0.0,0.0,0.0,6.0,5.0,2.2,1.378705,9.333333,2015,2014,1,11.911249,11,0
p100038,NaN,1.0,3.0,0.0,0.0,0.0,2.0,2.0,0.8,1.369306,2.000000,2015,2014,1,0.881829,4,0


In [23]:
def map_qua_year(x):
    if isinstance(x,str):
        if len(x)>3:
            return int(x.split('-')[0])
        else:
            return -1
    elif isinstance(x,float):
        if x!=NAN:
            return int(x)
        else:
            return -1

In [24]:
df_qualification['EXPIRYDATE']= df_qualification['EXPIRYDATE'].fillna(1)
df_qualification['BEGINDATE']= df_qualification['BEGINDATE'].fillna(1)
df_qualification['BEGINDATE']= df_qualification['BEGINDATE'].astype(str)
df_qualification['BEGINDATE']= df_qualification['BEGINDATE'].apply(lambda x:x.split('-')[0])
df_qualification['BEGINDATE'] =df_qualification['BEGINDATE'].astype(int)
df_qualification['EXPIRYDATE']= df_qualification['EXPIRYDATE'].apply(map_qua_year)

qua_bin = [0,2007,2008,2009,2010,2011,2012,2013,2014,2015]
qua_bin2 = [0,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,3000]
df_qualification['BEGINDATE'] = pd.cut(df_qualification['BEGINDATE'],qua_bin,labels=qua_bin[1:])
df_qualification['EXPIRYDATE'] = pd.cut(df_qualification['EXPIRYDATE'],qua_bin2,labels=qua_bin2[1:])

df_qua_table= pd.pivot_table(df_qualification,index=['EID'],columns =['ADDTYPE'],
                                         values=['BEGINDATE'],aggfunc={'BEGINDATE':len})
df_qua_type = pd.DataFrame(index=df_qua_table.index,columns=list(df_qua_table.columns.levels[1]),data=df_qua_table.values)

df_qua_vector = df_qua_type.fillna(0)
df_qua_vector['qua_var'] = df_qua_type.var(axis=1)
df_qua_vector['qua_count'] = df_qua_type.sum(axis=1)
df_qua_vector['qua_max'] = df_qua_type.max(axis=1)
df_qua_vector['qua_first_year'] = df_qualification.groupby(by=['EID'])['BEGINDATE'].min()
df_qua_vector['qua_last_year'] = df_qualification.groupby(by=['EID'])['BEGINDATE'].max()
df_qua_vector['qua_exp_first_year'] = df_qualification.groupby(by=['EID'])['EXPIRYDATE'].min()
df_qua_vector.to_csv('../data3/df_qualification_vector.csv')

In [25]:
df_qua_vector.head()

,1,2,3,qua_var,qua_count,qua_max,qua_first_year,qua_last_year,qua_exp_first_year
EID,,,,,,,,,
p100533,1.0,0.0,0.0,NaN,1.0,1.0,2015,2015,3000.0
p100847,1.0,0.0,0.0,NaN,1.0,1.0,2009,2009,2014.0
p10086,1.0,0.0,0.0,NaN,1.0,1.0,2013,2013,3000.0
p100916,0.0,1.0,3.0,2.0,4.0,3.0,2010,2014,2013.0
p101264,0.0,1.0,0.0,NaN,1.0,1.0,2014,2014,2017.0
